In [ ]:
import pandas as pd
import io
import os
from datetime import date,datetime,timedelta
import collections
from pathlib import Path
collections.Callable=collections.abc.Callable
import numpy as np


In [ ]:
os.chdir=(r"F:\Spectrum Auction")
print(os.getcwd())

In [ ]:
service_area_dict = {
'Andhra Pradesh': 'AP',
'Assam':'AS',
'Bihar':'BH',
'Delhi': 'DEL',
'Gujarat':'GU',
'Haryana':'HA',
'Himachal Pradesh': 'HP',
'Jammu & Kashmir': 'JK',
'Karnataka' : 'KA',
'Kerala': 'KE',
'Kolkata': 'KOL',
'Madhya Pradesh' : 'MP',
'Maharashtra': 'MA',
'Mumbai':'MU',
'North East' : 'NE',
'Odisha': 'OR',
'Punjab': 'PU',
'Rajasthan': 'RA',
'Tamil Nadu' : 'TN',
'Uttar Pradesh (East)' : 'UPE',
'Uttar Pradesh (West)' : 'UPW',
'West Bengal': 'WB'
            }

band_dict={'900(MHz)':900,
           '2100(MHz)': 2100,
           '2500(MHz)': 2500,
           '1800(MHz)': 1800}
block_size_dict={
    1800:0.2, 900:0.2,2100:5,2500:10}
bidder_dict={'Vodafone Idea Limited': 'Vi',
             'Bharti Airtel Limited': 'Airtel',
             'Reliance Jio Infocomm Limited': 'Jio'}


In [ ]:
df=pd.read_excel("Auction_Provisional_FAR.xlsx",engine='openpyxl',sheet_name="Sheet1")


In [ ]:
df=df.dropna(axis=1,how='all').dropna(axis=0,how='all').ffill().ffill(axis=1)

In [ ]:
df=df.replace(service_area_dict).replace(band_dict).replace(bidder_dict)

In [ ]:
df = df[df['Unnamed: 4'].str.match(r'^\d*\.?\d*$', na=True)]

In [ ]:
df.columns=["LSA","Band","Winning Price per block", "Bidder", "Uplink Start","Uplink Stop","Downlink Start","Downlink Stop"]

In [ ]:
set(df["Bidder"])

In [ ]:
new_row = {
    'LSA': 'BH',
    'Band': 2500,
    'Winning Price per block': 150.000,
    'Bidder': 'Vi',
    'Uplink Start': 2545,
    'Uplink Stop': 2555,
    'Downlink Start': None,  
    'Downlink Stop': None,    
    'Block Size': 10
}
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df["Block Size"]=[block_size_dict[x] for x in df["Band"]]

In [ ]:
df=df.replace(r'\*','',regex=True)

In [ ]:
df["MHz Won"] =df["Uplink Stop"].astype(float)-df["Uplink Start"].astype(float)

In [ ]:
df["Bidder Outflow"]= (df["MHz Won"]/df["Block Size"])*df["Winning Price per block"].astype(float)

In [ ]:
df.groupby(["Bidder"])["Bidder Outflow"].sum()

In [ ]:
gov_revenue=df.groupby(["Bidder"])["Bidder Outflow"].sum().sum()
print(gov_revenue)

In [ ]:
df.groupby(["LSA"])["MHz Won"].sum().sum()

In [ ]:
total_mhz_won =df.groupby(["LSA"])["MHz Won"].sum().sum()
print(total_mhz_won)

In [ ]:
df.groupby(['LSA','Band', 'Bidder'])['MHz Won'].sum()

In [ ]:
 df.groupby(['LSA','Band', 'Bidder'])['Bidder Outflow'].sum()

In [ ]:
total_rev= df.groupby(['LSA','Band', 'Bidder'])['Bidder Outflow'].sum().sum()
print(total_rev)